In [1]:
import numpy as np
import pandas as pd
from pyquery import PyQuery as pq
import time
import datetime
import requests
import urllib.request
import os
import calendar
import pickle

In [2]:
# input
ETF_data = pd.read_csv('Emerging Asia Pacific ETF List (114).csv')

# filter the inception date
ETF_data['Inception'] = pd.to_datetime(ETF_data['Inception'], format="%d/%m/%Y")
date = datetime.datetime(2016, 1, 1, 0, 0)
ETF_data = ETF_data.loc[ETF_data['Inception'] < date,:].reset_index()

# extract symbol and name of each ETF
symbol_list = ETF_data['Symbol']
name_list = ETF_data['ETF Name']

In [3]:
with open('ETF_dic.pkl','rb') as f:
    dic = pickle.load(f)

In [19]:
browser=webdriver.Chrome('/Users/lou/Downloads/chromedriver') 
browser.get('https://finance.yahoo.com/')
browser.find_element_by_name('yfin-usr-qry').send_keys(dic['name'][0]+'\n')
# link = browser.
browser.find_element_by_class_name('')
browser.quit()

In [ ]:
browser.find

In [54]:
s = dic['symbol'][0]
url = f'https://finance.yahoo.com/quote/{s}/history?p={s}'
options = webdriver.ChromeOptions()
prefs = {'download.default_directory': './data'}
options.add_experimental_option('prefs', prefs)

browser=webdriver.Chrome('/Users/lou/Downloads/chromedriver',options=options) 
browser.get(url)
option = browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
# option = browser.find_element_by_class_name('C($c-fuji-blue-1-b) Mstart(8px) Cur(p)')
start_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px)']")
print(start_date.get_attribute('value'))
start_date.clear()
start_date.send_keys('1/1/2016')
end_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px) Mstart(15px)']")
end_date.clear()
end_date.send_keys('12/31/2018')
browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()
browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Fl(end)']").click()
time.sleep(5)
browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
time.sleep(10)
url = browser.current_url
browser.quit()

3/15/2018


In [64]:
def get_all_symbol_csv(dic,url):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome('/Users/lou/Downloads/chromedriver',options=options) 
    
    for s in dic['symbol'][18:]:
        filepath = f'./data/{s}.csv'
        if os.path.isfile(filepath):
            print('File exist')
            continue
        url = f'https://finance.yahoo.com/quote/{s}/history?p={s}'
        browser.get(url)
        option = browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
        start_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px)']")
        start_date.clear()
        start_date.send_keys('1/1/2016')
        end_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px) Mstart(15px)']")
        end_date.clear()
        end_date.send_keys('12/31/2018')
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Fl(end)']").click()
        time.sleep(5)
        browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
#         time.sleep(10)
#         url = browser.current_url
#         browser.quit()

In [65]:
get_all_symbol_csv(dic,_)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']"}
  (Session info: chrome=73.0.3683.75)
  (Driver info: chromedriver=73.0.3683.68 (47787ec04b6e38e22703e856e101e840b65afe72),platform=Mac OS X 10.14.3 x86_64)
